In [115]:
import pandas as pd
import html
import re
import string


train_data = pd.read_csv('/Users/kevalpaida/Downloads/OLIDv1.0/olid-training-v1.0.tsv', sep='\t')
test_data = pd.read_csv('/Users/kevalpaida/Downloads/OLIDv1.0/testset-levela.tsv', sep='\t')
test_labels = pd.read_csv('/Users/kevalpaida/Downloads/OLIDv1.0/labels-levela.csv', header=None)

In [116]:
train_data.head(3)

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN


In [117]:
train_data = train_data.drop({'id','subtask_c','subtask_b'}, axis=1)
test_data = test_data.drop('id',axis=1)
train_data.head(3)

,tweet,subtask_a
0,@USER She should ask a few native Americans wh...,OFF
1,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF
2,Amazon is investigating Chinese employees who ...,NOT


In [118]:
#Preprocess

def process_tweet(csv):
    csv['tweet'] = csv.tweet.apply(lambda x: x.replace('@USER', ''))
    csv['tweet'] = csv.tweet.apply(lambda x: html.unescape(x))
    csv['tweet'] = csv['tweet'].str.replace(r'[^\x00-\x7F]+', '', regex=True)
    csv['tweet'] = csv['tweet'].str.replace('URL', '', regex=True)
     
    csv.rename(columns = {'tweet': 'sentence', 'subtask_a': 'label'}, inplace = True)
                                   
    return csv                              
                                   

In [133]:
def strip_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return text

def strip_all_entities(text):
    entity_prefixes = ['@','#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)


In [120]:
train = process_tweet(train_data)
test = process_tweet(test_data)

In [121]:
train['sentence'] = train['sentence'].apply(strip_links)
train['sentence'] = train['sentence'].apply(strip_all_entities)

test['sentence'] = test['sentence'].apply(strip_links)
test['sentence'] = test['sentence'].apply(strip_all_entities)

test['label'] = test_labels[1]

In [122]:
from sklearn import preprocessing
#le = preprocessing.LabelEncoder()

train['label'] = le.fit_transform(train['label'])
test['label'] = le.fit_transform(test['label'])

#"OFF" is 1 and "NOT" is 0

In [123]:
train.shape, test.shape

((13240, 2), (860, 2))

In [124]:
train.head()

,sentence,label
0,She should ask a few native Americans what the...,1
1,Go home youre drunk,1
2,Amazon is investigating Chinese employees who ...,0
3,Someone should veTaken this piece of shit to a...,1
4,Obama wanted liberals illegals to move into re...,0


In [125]:
test.head()

,sentence,label
0,Democrats Support Antifa Muslim Brotherhood MS...,1
1,is revered by Conservatives hated by Progressi...,0
2,First it reduces the ca,0
3,getting the news that she is still up for paro...,0
4,Unity demo to oppose the far right in Enough i...,1


In [126]:
from sklearn.model_selection import train_test_split

train, dev = train_test_split(train, test_size=0.10, random_state=42)

train.shape, dev.shape

((11916, 2), (1324, 2))

In [127]:
train.head(4)

,sentence,label
12937,broke the story therapist notes Miss Ford stat...,1
2448,Says the little commie thot I bet that 20 is m...,1
11275,Bet your butt,1
11986,You are so wrong Although I haven t had a Whop...,0


In [128]:
dev.head(4)

,sentence,label
12387,Why is John Kerry running his mouth again as i...,0
385,Gun control anyone,0
4896,What manga you reading,0
3971,You are right Victoria is on the reverse,0


In [134]:
train.label.value_counts()

0    7988
1    3928
Name: label, dtype: int64

In [136]:
dev.label.value_counts()

0    852
1    472
Name: label, dtype: int64

In [135]:
test.label.value_counts()

0    620
1    240
Name: label, dtype: int64

In [137]:
train.to_csv('/Users/kevalpaida/Documents/THESIS/SemEval6/data_new/train.tsv', sep='\t', index=False, header=True)
dev.to_csv('/Users/kevalpaida/Documents/THESIS/SemEval6/data_new/dev.tsv', sep='\t', index=False, header=True)
test.to_csv('/Users/kevalpaida/Documents/THESIS/SemEval6/data_new/test.tsv', sep='\t', index=False, header=True)




In [ ]:
#################_____-----END